# `join` no PySpark

In [0]:
spark

Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

## Criando o conjunto de dados

In [0]:
valoresA = [('Pedro',1),('Maria',2),('Ana',3),('Paulo',4)]
tabelaA = spark.createDataFrame(valoresA,['nome','id'])
 
valoresB = [('Mariana',1),('Pedro',2),('Ana',3),('Joao',4)]
tabelaB = spark.createDataFrame(valoresB,['nome','id'])

In [0]:
tabelaA.display()

nome,id
Pedro,1
Maria,2
Ana,3
Paulo,4


In [0]:
tabelaB.display()

nome,id
Mariana,1
Pedro,2
Ana,3
Joao,4


Para realizar o acesso as tabelas iremos criar um apelido (alias) para essas tabelas. O apelido, como no SQL, permite distinguir de onde uma determina coluna está vindo.

As colunas tem o mesmo nome, portanto, precisamos referenciar a tabelaA ou a tabelaB corretamente. O alias fornece um nome curto para referenciar os campos depois da criação da tabela com join.

In [0]:
ta = tabelaA.alias('ta')
tb = tabelaB.alias('tb')

Agora podemos utilizar ```ta.nome``` ou ```tb.nome```.

### Inner Join

Retorna as linhas que são comuns as duas tabelas.

É o tipo padrão para realizar o join.

In [0]:
df_inner_join = ta.join(tb, ta.nome == tb.nome, how='inner')
#df_inner_join = ta.join(tb, on=[nome])

In [0]:
df_inner_join.display()

nome,id,nome,id
Ana,3,Ana,3
Pedro,1,Pedro,2


Também é possível realizar o `join` via SQL

In [0]:
ta.createOrReplaceTempView('ta')
tb.createOrReplaceTempView('tb')

In [0]:
%sql

SELECT ta.nome, ta.id, tb.nome, tb.id 
     FROM ta 
     INNER JOIN tb 
     ON ta.nome = tb.nome

nome,id,nome,id
Ana,3,Ana,3
Pedro,1,Pedro,2


In [0]:
inner_join_sql = spark.sql(
  """SELECT ta.nome, ta.id, tb.nome, tb.id 
     FROM ta 
     INNER JOIN tb 
     ON ta.nome = tb.nome
  """
)
inner_join_sql.display()

nome,id,nome,id
Ana,3,Ana,3
Pedro,1,Pedro,2


In [0]:
%sql
SELECT ta.nome, ta.id, tb.nome, tb.id 
FROM ta 
INNER JOIN tb 
ON ta.nome = tb.nome

nome,id,nome,id
Ana,3,Ana,3
Pedro,1,Pedro,2


### Left Join

Retorna todos os registros que estão na tabela A (mesma que não estejam na tabela B) e os registros da tabela B que são comuns à tabela A (também pode ser utilizado o left_outer)

In [0]:
left_join = ta.join(tb, ta.nome == tb.nome, how='left')
left_join.display()

nome,id,nome,id
Pedro,1,Pedro,2
Maria,2,null,null
Ana,3,Ana,3
Paulo,4,null,null


In [0]:
left_outer_join = ta.join(tb, ta.nome == tb.nome, how='left_outer')
left_outer_join.display()

nome,id,nome,id
Pedro,1,Pedro,2
Maria,2,null,null
Ana,3,Ana,3
Paulo,4,null,null


Note que em ambos os casos o resultado inclui o ```null```. Podemos selecionar apenas os valores ```null``` ou não via método filter.

In [0]:
left_join.filter(left_join['tb.nome'].isNull()).display()
#left_join.filter(left_join.tb.nome.isNull()).display()
#left_join.filter(F.col('nome').isNull()).display()

nome,id,nome,id
Maria,2,null,null
Paulo,4,null,null


In [0]:
left_join.filter(left_join['tb.nome'].isNotNull()).display()

nome,id,nome,id
Ana,3,Ana,3
Pedro,1,Pedro,2


### Right Join

Retornar os registros que estão na tabela B (mesmo que não estejam na tabela A) e os registros da tabela A que são comuns à tabela B (também pode ser utilizado o right_outer).


In [0]:
right_join = ta.join(tb, ta.nome == tb.nome, how='right')
right_join.show()

+-----+----+-------+---+
| nome|  id|   nome| id|
+-----+----+-------+---+
| null|null|Mariana|  1|
|Pedro|   1|  Pedro|  2|
|  Ana|   3|    Ana|  3|
| null|null|   Joao|  4|
+-----+----+-------+---+



In [0]:
right_join.filter(right_join['ta.nome'].isNotNull()).show()

+-----+---+-----+---+
| nome| id| nome| id|
+-----+---+-----+---+
|  Ana|  3|  Ana|  3|
|Pedro|  1|Pedro|  2|
+-----+---+-----+---+



### Full Outer Join

Retorna todos os registros correspondentes de ambas as tabelas, independentemente de a outra tabela corresponder ou não. (também pode ser utilizado o full_outer ou outer).

In [0]:
full_outer_join = ta.join(tb, ta.nome == tb.nome,how='full') # Could also use 'full_outer'
full_outer_join.show()

+-----+----+-------+----+
| nome|  id|   nome|  id|
+-----+----+-------+----+
|  Ana|   3|    Ana|   3|
| null|null|   Joao|   4|
|Maria|   2|   null|null|
| null|null|Mariana|   1|
|Paulo|   4|   null|null|
|Pedro|   1|  Pedro|   2|
+-----+----+-------+----+



Esse comando irá mostrar todos as linhas da tabela da esquerda e todos as linhas da tabela da direita que correspondem a expressão e irá colocar ```null``` nas linhas que não correspondem.

### left_semi

Quando left_semi join é utilizado, todas as linhas do conjunto de dados da esquerda são retornados, caso tenham correspondência no conjunto de dados da direita.

Os dados finais contém apenas a informação (colunas) do conjunto de dados da esquerda.

In [0]:
left_semi = ta.join(tb, ta.nome == tb.nome, how='left_semi')
left_semi.show()

+-----+---+
| nome| id|
+-----+---+
|  Ana|  3|
|Pedro|  1|
+-----+---+



### left_anti

Retorna todas as linhas do conjunto de dados da esquerda que não correspondem ao conjunto de dados da direita.

In [0]:
left_anti = ta.join(tb, ta.nome == tb.nome, how='left_anti')
left_anti.show()

+-----+---+
| nome| id|
+-----+---+
|Maria|  2|
|Paulo|  4|
+-----+---+



### cross (ou cartesiano)

Combina cada linha no conjunto de dados da esquerda com cada linha no conjunto de dados da direita.

**Importante: Evite o uso do cartesiano, uma vez que vai explodir exponencialmente os dados. Muito provavelmente resultado em erro de memória.**

In [0]:
cross = ta.join(tb, ta.id == tb.id, how='cross')
cross.show()

+-----+---+-------+---+
| nome| id|   nome| id|
+-----+---+-------+---+
|Pedro|  1|Mariana|  1|
|Maria|  2|  Pedro|  2|
|  Ana|  3|    Ana|  3|
|Paulo|  4|   Joao|  4|
+-----+---+-------+---+




`[INFO] Fim do Notebook`